In [6]:
import read_write_model as colmap_reader
import numpy as np
import torch
import imageio.v3 as iio
import matplotlib.pyplot as plt
from datautils import read_data

%load_ext autoreload
%autoreload 2

In [7]:
imagedir = './images'
camFile = 'cameras.bin'
imgFile = 'images.bin'

images, c2w, H, W, F = read_data(camFile, imgFile, imagedir)

Loading Poses...
Done
Reading Images...
Done


In [46]:
i, j = torch.meshgrid(torch.arange(0, W, dtype = torch.int32), torch.arange(0, H, dtype=torch.int32), indexing = 'xy')
dirs = torch.stack([(i - W * 0.5) / F, -(j - H * .5)/F, -torch.ones_like(i)], -1)
rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[:, :3, :3], -1)
rays_o = torch.broadcast_to(torch.Tensor(c2w[:, :3, -1]), rays_d.shape)

ValueError: operands could not be broadcast together with shapes (2304,3072,1,3) (128,3,3) 

In [22]:
rays_o = torch.flatten(rays_o, start_dim = 0, end_dim = 1)
rays_d = torch.flatten(rays_d, start_dim = 0, end_dim = 1)

In [45]:
enc = PositionalEncoding()
posenc, direnc = enc.forward(rays_o, rays_d)
posenc.shape

torch.Size([7077888, 60])

In [44]:
class PositionalEncoding():
    def __init__(self, L_pos = 10, L_dir = 4):
        '''
        L_pos: Position (xyz) coordinates will be mapped to L_pos * 2 dimentions
        L_dir: Viewing direction unit vector will be mapped to L_dir * 2 dimentions
        '''
        self.L_pos = 10
        self.L_dir = 4

    def forward(self, pos, vdir):
        posenc = [] #Try including base pos and dir
        direnc = []
        
        for i in range(0, self.L_pos):
            for f in [torch.sin, torch.cos]:
                posenc.append(f(pos * np.pi * (2 ** i)))
                              
        for j in range(0, self.L_dir):
            for f in [torch.sin, torch.cos]:
                direnc.append(f(vdir * np.pi * (2 ** i)))
        
        posenc = torch.cat(posenc, -1)
        direnc = torch.cat(direnc, -1)
        
        return posenc, direnc
 